##  2. GLOSSÁRIO DE TERMOS E DICIONÁRIO DE DADOS

Este documento serve como a **referência única** para todos os termos técnicos, métricas de qualidade e indicadores estratégicos utilizados ao longo do pipeline de Engenharia de Dados (`src/`) e no Data Mart final (`data/outputs/`).

---

### 2.1. Métricas de Qualidade de Dados (ETL)

Fase crucial para garantir a confiabilidade dos dados transacionais brutos (`data/raw`) antes da análise.

| Termo | Definição | Base de Cálculo/Contexto |
| :--- | :--- | :--- |
| **Completude de CNPJ** | Percentual de registros com CNPJ válido (14 dígitos) após limpeza e remoção de caracteres inválidos. | `(df['cnpj_instituicao'] != '').mean() * 100` |
| **Taxa de Inversão Corrigida** | Percentual de registros onde foi detectada e corrigida a inversão de colunas (e.g., CNPJ no campo Nome) em dados históricos. | Métrica de qualidade do ETL para dados antigos. |
| **Encoding Detectado** | Codificação de caracteres identificada para leitura correta do arquivo. | Sequência de tentativas: `utf-8-sig, latin-1, iso-8859-1`. |
| **Taxa de Unicidade de Pedidos** | Percentual de registros com identificadores únicos após o processamento. | Exemplo: (263.259 IDs únicos / 263.562 registros totais = 99.9%) |
| **Limpeza Regional Brasileira** | Rotina específica para formatos monetários BR (R$, vírgula decimal, ponto milhar). | Conversão robusta da coluna `preco_unitario`. |

---

### 2.2. Técnicas e Heurísticas de Engenharia de Dados

Detalha as abordagens profissionais utilizadas para o *Data Wrangling* e a robustez do pipeline.

| Técnica | Descrição e Justificativa Técnica | Aplicação no Projeto |
| :--- | :--- | :--- |
| **Data Wrangling** | Termo profissional para a limpeza, padronização e transformação de dados brutos em um formato consistente e utilizável. | Aplicação de rotinas robustas de limpeza regional (moeda, data). |
| **Heurística de Dados** | Lógica baseada em regras (ex: contagem de caracteres alfabéticos vs. dígitos) para inferir e corrigir a qualidade de dados. | Usada para corrigir a **inversão de colunas** CNPJ/Nome em datasets históricos. |
| **dayfirst=True** | Parâmetro no `pd.to_datetime` que força a interpretação do formato de data como Dia/Mês/Ano. | Essencial para o tratamento de formatos de data regionais brasileiros. |
| **errors='coerce'** | Parâmetro no `pd.to_numeric` que transforma valores não convertíveis em `NaN`, preparando-os para o `fillna(0)`. | Essencial para o tratamento de valores sujos em colunas monetárias. |
| **Pipeline de Tolerância a Falhas** | ETL que continua processamento mesmo com erro em arquivos individuais. | Permite processar grandes lotes de arquivos (e.g., 2020-2022) mesmo se um falhar. |

---

### 2.3. Modelagem Dimensional (Star Schema)

Termos relacionados à estrutura do Data Mart final (`data/outputs/`), otimizada para consultas de BI.

| Termo | Descrição | Aplicação no Projeto |
| :--- | :--- | :--- |
| **Data Mart** | Um subconjunto de um Data Warehouse, focado em Compras Públicas e Risco de Aquisição. | O resultado final, consumido diretamente pelo Power BI. |
| **Star Schema** | Modelo de dados otimizado para consultas de BI, composto por uma Tabela Fato central e múltiplas Tabelas Dimensão. | Estrutura utilizada para o *Deployment* na pasta `data/outputs/`. |
| **Tabela Fato** | Tabela central com métricas transacionais (`Economia_por_Linha`, `preco_total`) e chaves estrangeiras. | `fato_compras_medicamentos.csv` (263.562 registros transacionais) |
| **Tabela Dimensão** | Tabela com atributos descritivos do negócio, separando dados repetitivos da Tabela Fato. | Exemplos: `dim_produto.csv`, `dim_instituicao.csv`, `dim_fornecedor.csv`. |
| **Chaves Surrogate** | Identificadores artificiais e mnemônicos (ex: `Ins00001`, `Pro00002`). | Garantia de unicidade e performance, facilitando a identificação da chave. |
| **Integridade Referencial** | Garantia de que todas as chaves estrangeiras na Tabela Fato possuem correspondência na Tabela Dimensão. | 100% das relações fato-dimensão validadas no *Deployment*. |

---

### 2.4. Feature Engineering: Indicadores de Risco (Risco e Governança)

Métricas estratégicas calculadas no módulo `modelagem_dim.py` para transformar transações em inteligência acionável.

| Termo | Definição | Priorização/Contexto |
| :--- | :--- | :--- |
| **PMP Mediano Dinâmico** | Preço Médio Ponderado Mediano de um produto por contexto (produto/ano). | **Benchmark robusto** para calcular desvios, mitigando o efeito de *outliers* extremos. |
| **Z-Score de Risco** (`score_z_risco`) | Medida estatística que quantifica a distância do preço pago em relação ao PMP Mediano, em desvios-padrão. | **Ranqueamento de Risco:** Valores acima de 2.0 ou 3.0 indicam um *outlier* de alto risco para auditoria. |
| **Desvio % Oportunidade** | Variação percentual do PMP pago por linha em relação ao PMP Mediano Dinâmico. | Quantifica a oportunidade de economia (ou sobrepreço) em termos percentuais. |
| **Economia por Linha** | Valor monetário (R$) do potencial de economia (ou prejuízo) calculado na transação. | **Priorização Gerencial:** Base para o **Gráfico de Pareto**, focando no maior retorno financeiro. |
| **Risco de Intermitência** | Métrica normalizada que mede a irregularidade na demanda de um produto (meses sem compra / período total). | Alerta para instabilidade na demanda, auxiliando na gestão de estoques. |
| **Risco de Concentração** | Mede a dependência do gasto total de um produto em relação ao seu fornecedor principal. | **Governança:** Alto valor indica risco de *lock-in* ou falta de concorrência. |
| **Índice de Priorização** | Métrica composta para ranquear oportunidades, combinando Risco de Preço, Risco de Intermitência e Concentração. | **Acionabilidade:** Permite ao gestor ranquear os produtos que exigem intervenção imediata. |

---

### 2.5. Orquestração e Deployment

Termos relacionados à automação da execução e à entrega final do Data Mart.

| Termo | Definição | Aplicação |
| :--- | :--- | :--- |
| **Orquestrador Central** (`main.py`) | O ponto de entrada único (*Entry Point*) do projeto, que coordena a execução sequencial de todos os módulos (ETL, Modelagem, Dimensões). | Garante a reprodutibilidade do pipeline completo e a automação do fluxo. |
| **Deployment (Carga Final)** | A fase final do pipeline, coordenada pelo `main.py`, que salva as tabelas Fato e Dimensão no formato pronto para consumo pelo Power BI. | Entrega dos ativos de dados para o ambiente de Business Intelligence. |
| **Flag analyze-only** | Parâmetro de linha de comando (`--apenas-analises`) que instrui o Orquestrador a carregar o arquivo consolidado existente e pular a fase demorada de ETL. | Otimiza o tempo de execução para testes de Modelagem (Desenvolvimento). |

---

### 2.6. Termos Específicos do Dataset

| Termo | Descrição | Uso no Projeto |
| :--- | :--- | :--- |
| **Código BR** | Código oficial de identificação do produto (OpenDataSUS ou fonte oficial). | Chave primária limpa e padronizada da Dimensão Produto. |
| **CATMAT** | Classificação Comum de Materiais (do SISMAD). | Usada nas dimensões para detalhar a descrição do produto. |
| **Mini Fato Radar** | Tabela auxiliar criada no *Feature Engineering* para armazenar as colunas pré-calculadas: **PMP Mediano**, **Z-Score** e **Economia por Linha**. | Usada para alimentar o Dashboard de Risco/Oportunidade. |
| **Drill-Through** | Funcionalidade do Power BI que permite ao usuário clicar em um ponto do visual e ser levado à página de investigação detalhada. | Focado na investigação do Fornecedor, Fabricante e Modalidade de Compra para o item selecionado. |
| **INDEFINIDO** | Valor de *fallback* (substituto) para categorias inválidas ou nulas. | Garante que colunas importantes (como `Tipo_Compra`) não contenham valores nulos. |

---

### 2.7. Métricas de Performance do Pipeline

| Termo | Definição | Importância |
| :--- | :--- | :--- |
| **Redução de Redundância** | Percentual de economia em armazenamento de dados descritivos (após criar Dimensões). | Ex: 87.8% de redução ao consolidar 1.3M+ instâncias em 32.127 registros dimensionais. |
| **Taxa de Sucesso por Arquivo** | Percentual de arquivos processados com sucesso. | Indicador de robustez e estabilidade do pipeline. |
| **Tempo de Processamento** | Duração do ETL desde leitura até consolidação. | Métrica chave para otimização de performance. |